# Group members

 - Flavia Morena Valentina Pacheco Jaime
 - Jozek Andrzej Hajduk Sánchez
 - Jorge Andres Rangel Zapata

# Connection with POSTGRESQL

In [2]:
import psycopg2
import pandas as pd

from google.cloud import bigquery
from google.oauth2 import service_account

In [3]:
conn = psycopg2.connect( host = 'localhost', port = 5432, database = 'Adventure', user = 'jorgedevops' )

# Getting the data

### Table 1 Employees per area

In [4]:
cur = conn.cursor()
table1 = pd.read_sql_query( 'SELECT st.name , count(pp.businessentityid) FROM sales.salesterritory st, sales.salesorderheader so, person.person pp WHERE pp.businessentityid = so.customerid AND st.territoryid  = so.territoryid group by (st.name)', conn )
cur.close()

_Data types table 1_

In [5]:
table1['name']= table1['name'].astype(str)
table1.dtypes

name     object
count     int64
dtype: object

In [6]:
table1

,name,count
0,France,1552
1,Southwest,3140
2,Northwest,2379
3,United Kingdom,2041
4,Northeast,7
5,Central,5
6,Australia,4283
7,Germany,1567
8,Southeast,13
9,Canada,2476


### Table 2 Units per subcategory

In [7]:
cur = conn.cursor()
table2 = pd.read_sql_query( 'SELECT  pp.name, count(ps.productsubcategoryid) FROM production.product pp, production.productsubcategory ps group by (pp.name)', conn )
cur.close()

In [8]:
table2

,name,count
0,LL Road Rim,37
1,"Mountain-100 Black, 38",37
2,"HL Road Frame - Black, 52",37
3,Flat Washer 6,37
4,Hex Nut 7,37
...,...,...
499,"HL Road Frame - Red, 48",37
500,"Road-350-W Yellow, 40",37
501,"LL Touring Frame - Blue, 58",37
502,"Men's Sports Shorts, M",37


### Table 3 Units per product

In [9]:
cur = conn.cursor()
table3 = pd.read_sql_query( 'SELECT PP.NAME, PI.QUANTITY FROM PRODUCTION.PRODUCT PP JOIN PRODUCTION.PRODUCTINVENTORY PI ON PP.PRODUCTID = PI.PRODUCTID;' ,conn)
cur.close()

In [10]:
table3

,name,quantity
0,Adjustable Race,408
1,Adjustable Race,324
2,Adjustable Race,353
3,Bearing Ball,427
4,Bearing Ball,318
...,...,...
1064,"Road-750 Black, 44",30
1065,"Road-750 Black, 48",99
1066,"Road-750 Black, 48",56
1067,"Road-750 Black, 52",78


### Table 4 Employees per department

In [11]:
cur = conn.cursor()
table4 = pd.read_sql_query( 'SELECT D.NAME, COUNT(ED.businessentityid) FROM HUMANRESOURCES.DEPARTMENT D JOIN HUMANRESOURCES.employeedepartmenthistory ED ON D.DEPARTMENTID = ED.businessentityid GROUP BY D.NAME;' ,conn)
cur.close()

In [12]:
table4

,name,count
0,Quality Assurance,1
1,Production Control,1
2,Research and Development,1
3,Sales,1
4,Document Control,1
5,Engineering,1
6,Executive,2
7,Purchasing,1
8,Finance,1
9,Facilities and Maintenance,1


# Connection with Google Cloud

## Credentials

In [13]:
credentials = service_account.Credentials.from_service_account_file( 
    '/home/jorge-arch/Descargas/sixth-foundry-314518-7cb450de31bf.json', 
    scopes = [ 'https://www.googleapis.com/auth/cloud-platform' ]
)

In [14]:
client = bigquery.Client( credentials = credentials, project = credentials.project_id )

### Table 1

In [15]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [16]:
job = client.load_table_from_dataframe(
    table1, 'Workshop.empleados_area', job_config = job_config
)
job.result()

### Table 2

In [17]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [18]:
job = client.load_table_from_dataframe(
    table2, 'Workshop.producto_por_categoria', job_config = job_config
)
job.result()

### Table 3

In [19]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'quantity', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

In [20]:
job = client.load_table_from_dataframe(
    table3, 'Workshop.productos_bogodega', job_config = job_config
)
job.result()

### Table 4

In [21]:
job_config = bigquery.LoadJobConfig(
    schema = [
        # Supported datatypes: https://cloud.google.com/bigquery/docs/reference/standard-sql/data-types
        bigquery.SchemaField( 'name', bigquery.enums.SqlTypeNames.STRING ),
        bigquery.SchemaField( 'count', bigquery.enums.SqlTypeNames.INT64 ),
    ],
    # Drod and re-create table, if exist
    write_disposition = 'WRITE_TRUNCATE',
)

job = client.load_table_from_dataframe(
    table4, 'Workshop.empleados_departamento', job_config = job_config
)
job.result()